In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
df = pd.read_csv("../input/breast-cancer-wisconsin-data/data.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df = df.drop(["Unnamed: 32", "id"], axis=1)

In [ ]:
df.describe().transpose()

# **Data Analysis**

In [ ]:
sns.countplot(x="diagnosis", data=df)

In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le = LabelEncoder()

In [ ]:
df["diagnosis"] = le.fit_transform(df["diagnosis"])

In [ ]:
le.classes_

In [ ]:
f, ax = plt.subplots(figsize=(16, 12))
plt.title('Pearson Correlation Matrix', fontsize=25)

sns.heatmap(df.corr(), linewidths=0.25, vmax=0.7, square=True, cmap="BuGn",
            linecolor='w', annot=True, annot_kws={"size":8}, cbar_kws={"shrink": .9});

area_mean, radius_mean and perimeter mean are correlated.

compactness_mean, concavity_mean and concave_points mean are correlated. Keep all for now.

radius_se, perimeter_se and area_se are correlated.

radius_worst, perimeter_worst and area_worst are correlated.

compactness_worst, concavity_worst and concave_points_worst are correlated. Keep all for now.

In [ ]:
#drop_features = ["radius_mean", "perimeter_mean", "perimeter_se", "area_se", "radius_worst", "area_worst"]
drop_features = ["radius_mean", "perimeter_mean", "perimeter_se", "radius_se", "radius_worst", "perimeter_worst"]
#drop_features = ["area_mean", "perimeter_mean", "area_se", "perimeter_se", "perimeter_worst", "area_worst"]


In [ ]:
df = df.drop(drop_features, axis=1)

In [ ]:
df.head()

In [ ]:
df.corr()["diagnosis"].sort_values().plot(kind="bar")

# **Train Validation Test Split**

In [ ]:
X = df.drop("diagnosis", axis=1).values
y = df["diagnosis"].values

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=0)

60% train, 20% validation and 20% test

In [ ]:
X_train.shape

In [ ]:
X_val.shape

In [ ]:
X_test.shape

# **Scaling the Data**

In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler = MinMaxScaler()

In [ ]:
X_train = scaler.fit_transform(X_train)

In [ ]:
X_val = scaler.transform(X_val)

In [ ]:
X_test = scaler.transform(X_test)

# **Model**

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout

In [ ]:
model = Sequential()

model.add(Dense(24, activation="relu"))
model.add(Dropout(0.2))

model.add(Dense(16, activation="relu"))
model.add(Dropout(0.5))

model.add(Dense(1, activation="sigmoid"))

model.compile(loss="binary_crossentropy", optimizer="adam")

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [ ]:
check_point = ModelCheckpoint("best_model.h5", monitor="val_loss", verbose=1, save_best_only=True)

In [ ]:
early_stop = EarlyStopping(monitor="val_loss", patience=1000)

In [ ]:
model.fit(x=X_train, y=y_train,
          epochs=10000, callbacks=[check_point, early_stop],
          validation_data=(X_val,y_val), verbose=1)

In [ ]:
losses = pd.DataFrame(model.history.history)

In [ ]:
losses.plot()

In [ ]:
losses["val_loss"].min()

In [ ]:
from keras.models import load_model

In [ ]:
saved_model = load_model('best_model.h5')

In [ ]:
predictions = saved_model.predict_classes(X_test)

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(classification_report(y_test,predictions))

In [ ]:
print(confusion_matrix(y_test,predictions))